In [6]:
import pandas as pd
import sqlite3
import os

In [7]:
#Path
CSV_PATH = "../data/SuperstoreData.csv" 
DB_PATH  = "../sql/superstore.db"

In [8]:
#Load & CLean Data
df = pd.read_csv(CSV_PATH, encoding="latin1")

df.head()

,Row ID,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Customer Name,Segment,Country,City,...,Postal Code,Region,Product ID,Category,Sub-Category,Product Name,Sales,Quantity,Discount,Profit
0,1,CA-2016-152156,11/8/2016,11/11/2016,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,...,42420,South,FUR-BO-10001798,Furniture,Bookcases,Bush Somerset Collection Bookcase,261.9600,2,0.00,41.9136
1,2,CA-2016-152156,11/8/2016,11/11/2016,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,...,42420,South,FUR-CH-10000454,Furniture,Chairs,"Hon Deluxe Fabric Upholstered Stacking Chairs,...",731.9400,3,0.00,219.5820
2,3,CA-2016-138688,6/12/2016,6/16/2016,Second Class,DV-13045,Darrin Van Huff,Corporate,United States,Los Angeles,...,90036,West,OFF-LA-10000240,Office Supplies,Labels,Self-Adhesive Address Labels for Typewriters b...,14.6200,2,0.00,6.8714
3,4,US-2015-108966,10/11/2015,10/18/2015,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,...,33311,South,FUR-TA-10000577,Furniture,Tables,Bretford CR4500 Series Slim Rectangular Table,957.5775,5,0.45,-383.0310
4,5,US-2015-108966,10/11/2015,10/18/2015,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,...,33311,South,OFF-ST-10000760,Office Supplies,Storage,Eldon Fold 'N Roll Cart System,22.3680,2,0.20,2.5164


In [ ]:
for col in ["Sales", "Profit", "Discount", "Quantity"]:
    if col in df.columns:
        df[col] = pd.to_numeric(df[col], errors="coerce")

# Convert 'Order Date' to datetime
df["Order Date"] = pd.to_datetime(df["Order Date"], format="%m/%d/%Y", errors='coerce')
# Convert 'Ship Date' to datetime
df["Ship Date"] = pd.to_datetime(df["Ship Date"], format="%m/%d/%Y", errors='coerce')


# Helper columns for SQL analytics
df["YearMonth"]    = df["Order Date"].dt.strftime("%Y-%m")      # text is easiest in SQLite
df["OrderDate"]    = df["Order Date"].dt.strftime("%Y-%m-%d")
df["ShipDate"]     = df["Ship Date"].dt.strftime("%Y-%m-%d")
df["ShippingDays"] = (pd.to_datetime(df["Ship Date"]) - pd.to_datetime(df["Order Date"])).dt.days
df["ProfitMargin"] = (df["Profit"] / df["Sales"]).replace([pd.NA, float("inf"), -float("inf")], 0)


In [ ]:
# Write to SQLite ===
db_path = "../sql/superstore.db"
conn = sqlite3.connect(db_path)

# (Re)create table
df.to_sql("orders", conn, if_exists="replace", index=False)

# Indexes for speed
conn.executescript("""
DROP INDEX IF EXISTS idx_orders_orderdate;
DROP INDEX IF EXISTS idx_orders_customers;
DROP INDEX IF EXISTS idx_orders_region;
DROP INDEX IF EXISTS idx_orders_category;

CREATE INDEX idx_orders_orderdate ON orders(OrderDate);
CREATE INDEX idx_orders_customers ON orders("Customer Name");
CREATE INDEX idx_orders_region ON orders(Region);
CREATE INDEX idx_orders_category ON orders(Category);
""")

# === Views ===
schema_sql = """
DROP VIEW IF EXISTS monthly_kpis;
CREATE VIEW monthly_kpis AS
SELECT
  YearMonth,
  COUNT(*)                         AS orders_count,
  SUM(Sales)                       AS total_sales,
  SUM(Profit)                      AS total_profit,
  AVG(Discount)                    AS avg_discount,
  CASE WHEN SUM(Sales) <> 0 THEN SUM(Profit) * 1.0 / SUM(Sales) ELSE 0 END AS profit_margin
FROM orders
GROUP BY YearMonth
ORDER BY YearMonth;

DROP VIEW IF EXISTS top_customers;
CREATE VIEW top_customers AS
SELECT
  "Customer Name"                  AS customer_name,
  COUNT(*)                         AS orders_count,
  SUM(Sales)                       AS total_sales,
  SUM(Profit)                      AS total_profit,
  AVG(Discount)                    AS avg_discount
FROM orders
GROUP BY "Customer Name"
ORDER BY total_sales DESC;

DROP VIEW IF EXISTS discount_profit_curve;
CREATE VIEW discount_profit_curve AS
WITH buckets AS (
  SELECT
    CAST(Discount * 10 AS INT) / 10.0 AS discount_bucket,
    Profit
  FROM orders
)
SELECT 
  discount_bucket,
  COUNT(*)                 AS orders_count,
  AVG(Profit)              AS avg_profit,
  SUM(Profit)              AS total_profit
FROM buckets
GROUP BY discount_bucket
ORDER BY discount_bucket;

DROP VIEW IF EXISTS region_performance;
CREATE VIEW region_performance AS
SELECT 
  Region,
  COUNT(*)                  AS orders_count,
  SUM(Sales)                AS total_sales,
  SUM(Profit)               AS total_profit,
  AVG(ShippingDays)         AS avg_shipping_days
FROM orders
GROUP BY Region
ORDER BY total_sales DESC;

DROP VIEW IF EXISTS loss_orders_by_category;
CREATE VIEW loss_orders_by_category AS
SELECT
  Category,
  COUNT(*) AS loss_orders
FROM orders
WHERE Profit < 0
GROUP BY Category
ORDER BY loss_orders DESC;

DROP VIEW IF EXISTS shipping_performance;
CREATE VIEW shipping_performance AS
WITH stats AS (
  SELECT
    Region,
    CASE WHEN ShippingDays > 3 THEN 1 ELSE 0 END AS is_late
  FROM orders
)
SELECT

  Region,
  AVG(is_late) * 100.0 AS pct_late_shipments
FROM stats
GROUP BY Region
ORDER BY pct_late_shipments DESC;
"""
conn.executescript(schema_sql)

#  Example exports (nice for Power BI or your repo) ===
def dump(query, out_csv):
    pd.read_sql_query(query, conn).to_csv(out_csv, index=False)

dump("SELECT * FROM monthly_kpis;",             "sql_monthly_kpis.csv")
dump("SELECT * FROM top_customers LIMIT 25;",   "sql_top_customers_top25.csv")
dump("SELECT * FROM discount_profit_curve;",    "sql_discount_profit_curve.csv")
dump("SELECT * FROM region_performance;",       "sql_region_performance.csv")
dump("SELECT * FROM loss_orders_by_category;",  "sql_loss_orders_by_category.csv")
dump("SELECT * FROM shipping_performance;",     "sql_shipping_performance.csv")
conn.close()

print("SQLite database created:: superstore.sqlite")
print("CSV exports written.................................................)

NameError: name 'sqlite3' is not defined